In [5]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import requests
from requests.exceptions import HTTPError
import json
from typing import List, Set

class Question:
    def __init__(self, id: int, titleSlug: str, url: str=''):
        self.id = id
        self.titleSlug = titleSlug
        self.url = f"https://leetcode.com/problems/{titleSlug}/"
    
    def __str__(self):
        return f"[{self.id}. {self.titleSlug}]"
    
    def __repr__(self):
        return f"{self.id}: {self.url}"
    
    def __hash__(self):
        return self.id
    
    def __eq__(self, other):
        return isinstance(other, Question) and other.id == self.id and other.titleSlug == self.titleSlug

class LeetcodeClient:
    _transport = RequestsHTTPTransport(
        url='https://leetcode.com/graphql',
        use_json=True,
    )
    
    _client = Client(
        transport=_transport,
        fetch_schema_from_transport=True,
    )
    
    _questionCache = {}
    
    def getSimilarQuestions(self, title: str) -> List[Question]:
        query = gql("""
            query getQuestionDetail($titleSlug: String!) {
              question(titleSlug: $titleSlug) {
                similarQuestions
              }
            }
            """)
        params = {"titleSlug": title}
        json_dict = LeetcodeClient._client.execute(query, variable_values=params)
        if not json_dict:
            return []
        similar_qstr = json_dict['question']['similarQuestions']
        similar_qarr = json.loads(similar_qstr)
        return [Question(self.getQuestionId(q['titleSlug']), q['titleSlug']) for q in similar_qarr]

    def toTitle(self, questions: List[Question]) -> List[str]:
        return map(lambda x: x.titleSlug, curset)
    
    def getSimilarQuestionCluster(self, question: Question) -> List[Question]:
        q = self.getSimilarQuestions(question.titleSlug)
        visited = set(q+[question])
        depth = 3
        h = 0
        while q and h < depth:
            l = len(q)
            h += 1
            for _ in range(l):
                cur_title = q.pop(0).titleSlug
                new_q = self.getSimilarQuestions(cur_title)
                new_questions = list(set(new_q) - visited)
                new_qset = list(map(lambda x: x.titleSlug, new_questions))
#                 print(f"{cur_title} -> {new_qset}")
                q.extend(new_questions)
                visited |= set(new_q)
        return sorted(list(visited), key=lambda x: x.id)
    
    def getlAllQuestions(self) -> List[Question]:
        url = 'https://leetcode.com/api/problems/all'
        try:
            response = requests.get(url)
            # If the response was successful, no Exception will be raised
            response.raise_for_status()
            result = response.json()
            questions = result['stat_status_pairs']
            res = []
            for ele in questions:
                q = Question(ele['stat']['question_id'], ele['stat']['question__title_slug'])
                LeetcodeClient._questionCache[q.titleSlug] = q
                res.append(q)
            return sorted(res, key=lambda q: q.id)
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')  # Python 3.6
        except Exception as err:
            print(f'Other error occurred: {err}')  # Python 3.6
        else:
            print('Success!')
        return []
    
    def getQuestionId(self, title: str) -> int:
        default = Question(0, '')
        return LeetcodeClient._questionCache.get(title, default).id

In [11]:
lc_client = LeetcodeClient()
questions = lc_client.getlAllQuestions()

In [13]:
# title = 'reconstruct-itinerary'
title = 'valid-palindrome-iii'
lc_client.getSimilarQuestions(title)

[]

In [8]:
q = Question(lc_client.getQuestionId(title), title)
lc_client.getSimilarQuestionCluster(q)

[332: https://leetcode.com/problems/reconstruct-itinerary/]

In [9]:
from time import sleep

questions = lc_client.getlAllQuestions()
cache = {q.titleSlug:q for q in questions}
visited = set()
for q in questions:
    if q.titleSlug not in visited:
        qcluster = lc_client.getSimilarQuestionCluster(q)
        for subq in qcluster:
            print(subq)
        print("")
        sleep(2)
        visited |= set(lc_client.toTitle(qcluster))

[1. two-sum]
[15. 3sum]
[16. 3sum-closest]
[18. 4sum]
[53. maximum-subarray]
[78. subsets]
[152. maximum-product-subarray]
[167. two-sum-ii-input-array-is-sorted]
[170. two-sum-iii-data-structure-design]
[198. house-robber]
[209. minimum-size-subarray-sum]
[238. product-of-array-except-self]
[259. 3sum-smaller]
[288. unique-word-abbreviation]
[303. range-sum-query-immutable]
[320. generalized-abbreviation]
[325. maximum-size-subarray-sum-equals-k]
[411. minimum-unique-word-abbreviation]
[454. 4sum-ii]
[523. continuous-subarray-sum]
[525. contiguous-array]
[560. subarray-sum-equals-k]
[611. valid-triangle-number]
[628. maximum-product-of-three-numbers]
[653. two-sum-iv-input-is-a-bst]
[713. subarray-product-less-than-k]
[724. find-pivot-index]
[1016. subarray-sums-divisible-by-k]
[1083. two-sum-less-than-k]
[1150. two-sum-bsts]



NameError: name 'curset' is not defined